# One Score to Rule Them All
The time finally comes to assess which neighborhood might be worthy of the title of best! We decided to give each neighborhood a score out of 100 in order to simplify the process.

Each member of the team decided on an equation to give the neighborhoods a score, based on their individual metric.

In [13]:
import pandas as pd
import geopandas as gp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [43]:
#EBLL score
nb = gp.read_file("neighborhoods/Neighborhoods_.shp")
ell = pd.read_csv("elevated-blood-lead-levels.csv")
nb.insert(0,"tract",nb['tractce10'].astype("Int64", errors='ignore').fillna(0))
ell.insert(0,"tract",(ell["CensusTract"]-42003000000))
ellnb = nb.merge(ell, how="left", on="tract")
scored = ellnb[['hood','percentEBLL15_20']].dropna().sort_values('percentEBLL15_20')
scored.insert(2,'EBLLscore',100-ezview['percentEBLL15_20']*10)
EBLLscored = scored[['hood','EBLLscore']]

,hood,percentEBLL15_20,EBLLscore
1,Summer Hill,0.00,100.0
24,Bloomfield,0.00,100.0
38,West Oakland,0.00,100.0
20,New Homestead,0.00,100.0
40,Strip District,0.00,100.0
54,Swisshelm Park,0.00,100.0
61,South Side Flats,0.00,100.0
19,Lincoln Place,0.48,95.2
35,East Liberty,0.49,95.1
51,Stanton Heights,0.64,93.6


In [21]:
#Education score
school_data = pd.read_csv('school-enrollment-by-detailed-level-of-school-for-the-population-3-years-and-over.csv')#open the file to read
percent_list = []
neigh_list = []
score_list = []

for index, row in school_data.iterrows():# go through each row 
    percent_list.append(100 * (row[4]/row[2]))     # puts percent into percent list  
    neigh_list.append(row[0])     # put neighborhood in
    
d = {'Neighborhood': neigh_list, 'Education Score of Neighborhood': percent_list}#create new data set with neighborhoods and percentages
percent_list = pd.DataFrame(data=d)# creates a new df so we can get the right scale for the points

for index, row in percent_list.iterrows():# go through each row 
    score_list.append(100 * (row[1]/percent_list["Education Score of Neighborhood"].max()))# append new score scale to score_list 

d = {'Neighborhood': neigh_list, 'Education Score of Neighborhood': score_list}#create new data set with neighborhoods and percentages
eduscored = pd.DataFrame(data=d)

In [44]:
police_arrest = pd.read_csv("police_arrest.csv")
filter1 = police_arrest["OFFENSES"].str.contains("dui", case = False, na = False)
filter2 = police_arrest["OFFENSES"].str.contains("assault", case = False, na = False)
p = police_arrest[filter1 | filter2]
percent = (p["INCIDENTNEIGHBORHOOD"].value_counts()/police_arrest["INCIDENTNEIGHBORHOOD"].value_counts()) * 100
percent.fillna(0, inplace = True)
score = 100 - percent
lawscored = score.sort_values()
ls = lawscored.to_frame().reset_index()

In [45]:
merged = EBLLscored.merge(eduscored, how='outer', left_on='hood', right_on='Neighborhood')
merged = merged.merge(ls, how='outer', left_on='hood', right_on='index').fillna(0)
merged.insert(0,'finalscore',merged['EBLLscore']*.4+merged['Education Score of Neighborhood']*.2+merged['INCIDENTNEIGHBORHOOD']*.4)
merged[['hood','finalscore']].sort_values('finalscore')

,hood,finalscore
83,0,0.000000
81,0,3.192586
85,0,3.959412
88,0,5.523159
89,0,5.904673
...,...,...
8,East Liberty,74.626083
1,Bloomfield,74.946938
13,Central Business District,75.262341
6,South Side Flats,75.310536
